In [1]:
import petpy
from config import API_key, API_secret
import json
import pymongo
from petpy import Petfinder
import pandas as pd

In [2]:
pf = Petfinder(key=API_key, secret=API_secret)

In [3]:
# animals = pf.animals() - see below for query with edited results limits

In [4]:
animals = pf.animals(results_per_page=100, pages=100)

In [5]:
animals

{'animals': [{'id': 50353495,
   'organization_id': 'OH383',
   'url': 'https://www.petfinder.com/cat/angel-and-paris-bonded-pair-50353495/oh/cincinnati/three-sisters-pet-rescue-ohio-hound-rescue-oh383/?referrer_id=cd5242ea-963a-4b60-8b2c-b458aeb14609',
   'type': 'Cat',
   'species': 'Cat',
   'breeds': {'primary': 'Calico',
    'secondary': 'Dilute Calico',
    'mixed': True,
    'unknown': False},
   'colors': {'primary': 'Calico',
    'secondary': 'Dilute Calico',
    'tertiary': None},
   'age': 'Adult',
   'gender': 'Female',
   'size': 'Medium',
   'coat': 'Short',
   'attributes': {'spayed_neutered': True,
    'house_trained': True,
    'declawed': False,
    'special_needs': False,
    'shots_current': True},
   'environment': {'children': None, 'dogs': None, 'cats': True},
   'tags': ['Loving', 'Social', 'Playful'],
   'name': 'Angel & Paris, Bonded Pair',
   'description': 'Cats listed here have entered our program and have received a basic level of vet care including: vacci

In [6]:
# find number of animal_dicts in the list within the overal animals dict
# animals["animals"][0]
# len(animals["animals"][0])
len(animals["animals"])

10000

In [7]:
conn = "mongodb://localhost:27017"
client = pymongo.MongoClient(conn)

In [8]:
db = client.rescue_angels_db

In [9]:
animals_collection = db.animals_collection

In [10]:
for animal in animals["animals"]:
    animal_dict = animal
    animals_collection.insert_one(animal_dict)

In [11]:
# organizations = pf.organizations(results_per_page=100, pages=100) returns a duplicate record error when 
# trying to send it to mongo. Try getting organizations into pandas to look at duplicates 
organizations_df = pf.organizations(results_per_page=100, pages=100, return_df=True)

/Users/chloe/opt/anaconda3/envs/PythonData/lib/python3.6/site-packages/petpy/api.py:1049: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  results_df = json_normalize(results[key])


In [12]:
organizations_df

,id,name,email,phone,url,website,mission_statement,photos,distance,address.address1,...,hours.saturday,hours.sunday,adoption.policy,adoption.url,social_media.facebook,social_media.twitter,social_media.youtube,social_media.instagram,social_media.pinterest,organization_id
0,PA912,The Shelter Animal Project,tsapadoption@gmail.com,None,https://www.petfinder.com/member/us/pa/fairles...,None,None,[{'small': 'https://dl5zpyw5k3jeb.cloudfront.n...,None,PO Box 24,...,None,None,None,None,None,None,None,None,None,pa912
1,GA762,Special Touch Animal Rescue,STARRadopt@aol.com,(404) 983-3581,https://www.petfinder.com/member/us/ga/acworth...,None,We are a private foster home group. We are NOT...,[],None,None,...,None,None,None,None,None,None,None,None,None,ga762
2,TX876,Gina's Heart of Gold Reptile Rescue,gdisteldorf@comcast.net,(713) 329-9304,https://www.petfinder.com/member/us/tx/houston...,None,None,[],None,None,...,None,None,None,None,None,None,None,None,None,tx876
3,OH749,Purr-fect Companions Sanctuary,purrfectcompanions@att.net,216-671-6369,https://www.petfinder.com/member/us/oh/lakewoo...,None,None,[],None,P.O.Box 770992,...,None,None,None,None,None,None,None,None,None,oh749
4,MO556,Montgomery City Animal Shelter/Project Preciou...,projectpreciouspaws@hotmail.com,1-573-310-4979,https://www.petfinder.com/member/us/mo/montgom...,None,None,[{'small': 'https://dl5zpyw5k3jeb.cloudfront.n...,None,915 Benton Street,...,None,None,None,None,None,None,None,None,None,mo556
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,NJ519,Close to Home Rescue,Doggiedogrescue@aol.com,None,https://www.petfinder.com/member/us/nj/pompton...,https://www.doggiedogworld.petfinder.com,None,[{'small': 'https://dl5zpyw5k3jeb.cloudfront.n...,None,None,...,None,None,We are foster based and do not have a physical...,None,https://www.facebook.com/closetohome/,None,None,None,None,nj519
9996,CT477,Pack Leaders Dog Rescue of CT,packleadersrescueCT@gmail.com,None,https://www.petfinder.com/member/us/ct/manches...,http://www.packleadersrescue.com,UPCOMING ADOPTION EVENTS\r\nhttp://packleaders...,[{'small': 'https://dl5zpyw5k3jeb.cloudfront.n...,None,None,...,None,None,Please understand that our Rescue dogs have be...,http://packleadersrescue.com/adoption-applicat...,None,None,None,None,None,ct477
9997,KY544,"My New Kentucky Home, Inc.",mynewkentuckyhomerescue@gmail.com,(502) 509-3848,https://www.petfinder.com/member/us/ky/louisvi...,None,Our mission is to help our overcrowded Kentuck...,[{'small': 'https://dl5zpyw5k3jeb.cloudfront.n...,None,None,...,None,None,None,None,https://facebook.com/MyNewKentuckyHomeRescue,None,None,None,None,ky544
9998,OK151,Oklahoma Westie Rescue,adoption@okwestierescue.com,None,https://www.petfinder.com/member/us/ok/bixby/o...,None,Oklahoma Westie Rescue is dedicated to the res...,[{'small': 'https://dl5zpyw5k3jeb.cloudfront.n...,None,None,...,None,None,NOTE: You must complete and submit an applicat...,None,None,None,None,None,None,ok151


In [13]:
unique_ids = organizations_df["id"].unique()
print (unique_ids)

['PA912' 'GA762' 'TX876' ... 'KY544' 'OK151' 'CA1403']


In [14]:
# checking list length
len_org = len(organizations_df)
len_org

10000

In [15]:
# checking number of unique names
len_name_unique = len(organizations_df["name"].unique())
len_name_unique

8885

In [16]:
# checking number of unique ids (column 1)
len_id_unique = len(organizations_df["id"].unique())
len_id_unique

9030

In [17]:
# checking number of unique organization_ids (column 30)
len_orgID_unique = len(organizations_df["organization_id"].unique())
len_orgID_unique

9030

In [18]:
# looking for instances of repeated ids
organizations_df["id"].value_counts()

NJ968     3
ME158     3
ME150     3
MI1011    3
AL227     3
         ..
LA314     1
NC647     1
NJ924     1
NY1181    1
OH759     1
Name: id, Length: 9030, dtype: int64

In [19]:
# check one of the duplicate record types to inspect. -- note that all the fields seem to match
#CT190 = organizations_df.loc[organizations_df["id"] == "CT190"]
#CT190

In [20]:
# checking ot see if creating a df in a different way would 

#CT190_names = CT190[["id","name"]]
#CT190_names

In [21]:
# CA2698_names2 = pd.options.display.max_colwidth

In [22]:
#CT190_names.iloc[0, 1]

In [23]:
#CT190_names.iloc[1, 1]

In [24]:
#CT190_names.iloc[2, 1]

In [25]:
# organizations2_df = organizations_df.drop_duplicates() throws and error. 
# photos field is creating an error in pandas because it contains a list within the field. 
# create new df without the photos field. first, get all column headers

column_list = list(organizations_df.columns) 
column_list

['id',
 'name',
 'email',
 'phone',
 'url',
 'website',
 'mission_statement',
 'photos',
 'distance',
 'address.address1',
 'address.address2',
 'address.city',
 'address.state',
 'address.postcode',
 'address.country',
 'hours.monday',
 'hours.tuesday',
 'hours.wednesday',
 'hours.thursday',
 'hours.friday',
 'hours.saturday',
 'hours.sunday',
 'adoption.policy',
 'adoption.url',
 'social_media.facebook',
 'social_media.twitter',
 'social_media.youtube',
 'social_media.instagram',
 'social_media.pinterest',
 'organization_id']

In [26]:
organizations2_df = organizations_df[['id',
 'name',
 'email',
 'phone',
 'url',
 'website',
 'mission_statement',
 'distance',
 'address.address1',
 'address.address2',
 'address.city',
 'address.state',
 'address.postcode',
 'address.country',
 'hours.monday',
 'hours.tuesday',
 'hours.wednesday',
 'hours.thursday',
 'hours.friday',
 'hours.saturday',
 'hours.sunday',
 'adoption.policy',
 'adoption.url',
 'social_media.facebook',
 'social_media.twitter',
 'social_media.youtube',
 'social_media.instagram',
 'social_media.pinterest',
 'organization_id']]

In [27]:
# rename fieldnames containing a dot, as this was throwing an error trying to send to mongo 
organizations2_df = organizations2_df.rename(columns = {
 'address.address1': "address_address1",
 'address.address2': "address_address2",
 'address.city': "address_city" ,
 'address.state': "address_state",
 'address.postcode': "address_postcode",
 'address.country': "address_country",
 'hours.monday': "hours_monday",
 'hours.tuesday': "hours_tuesday",
 'hours.wednesday': "hours_wednesday",
 'hours.thursday': "hours_thursday",
 'hours.friday': "hours_friday",
 'hours.saturday': "hours_saturdsy",
 'hours.sunday': "hours_sunday",
 'adoption.policy': "adoption_policy" ,
 'adoption.url': "adoption_url",
 'social_media.facebook': "socialmedia_facebook",
 'social_media.twitter': "socialmedia_twitter",
 'social_media.youtube': "socialmedia_youtube",
 'social_media.instagram': "socialmedia_instagram",
 'social_media.pinterest': "socialmedia_pinterest",
})


In [28]:
organizations2_df

,id,name,email,phone,url,website,mission_statement,distance,address_address1,address_address2,...,hours_saturdsy,hours_sunday,adoption_policy,adoption_url,socialmedia_facebook,socialmedia_twitter,socialmedia_youtube,socialmedia_instagram,socialmedia_pinterest,organization_id
0,PA912,The Shelter Animal Project,tsapadoption@gmail.com,None,https://www.petfinder.com/member/us/pa/fairles...,None,None,None,PO Box 24,None,...,None,None,None,None,None,None,None,None,None,pa912
1,GA762,Special Touch Animal Rescue,STARRadopt@aol.com,(404) 983-3581,https://www.petfinder.com/member/us/ga/acworth...,None,We are a private foster home group. We are NOT...,None,None,None,...,None,None,None,None,None,None,None,None,None,ga762
2,TX876,Gina's Heart of Gold Reptile Rescue,gdisteldorf@comcast.net,(713) 329-9304,https://www.petfinder.com/member/us/tx/houston...,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,tx876
3,OH749,Purr-fect Companions Sanctuary,purrfectcompanions@att.net,216-671-6369,https://www.petfinder.com/member/us/oh/lakewoo...,None,None,None,P.O.Box 770992,None,...,None,None,None,None,None,None,None,None,None,oh749
4,MO556,Montgomery City Animal Shelter/Project Preciou...,projectpreciouspaws@hotmail.com,1-573-310-4979,https://www.petfinder.com/member/us/mo/montgom...,None,None,None,915 Benton Street,None,...,None,None,None,None,None,None,None,None,None,mo556
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,NJ519,Close to Home Rescue,Doggiedogrescue@aol.com,None,https://www.petfinder.com/member/us/nj/pompton...,https://www.doggiedogworld.petfinder.com,None,None,None,None,...,None,None,We are foster based and do not have a physical...,None,https://www.facebook.com/closetohome/,None,None,None,None,nj519
9996,CT477,Pack Leaders Dog Rescue of CT,packleadersrescueCT@gmail.com,None,https://www.petfinder.com/member/us/ct/manches...,http://www.packleadersrescue.com,UPCOMING ADOPTION EVENTS\r\nhttp://packleaders...,None,None,None,...,None,None,Please understand that our Rescue dogs have be...,http://packleadersrescue.com/adoption-applicat...,None,None,None,None,None,ct477
9997,KY544,"My New Kentucky Home, Inc.",mynewkentuckyhomerescue@gmail.com,(502) 509-3848,https://www.petfinder.com/member/us/ky/louisvi...,None,Our mission is to help our overcrowded Kentuck...,None,None,None,...,None,None,None,None,https://facebook.com/MyNewKentuckyHomeRescue,None,None,None,None,ky544
9998,OK151,Oklahoma Westie Rescue,adoption@okwestierescue.com,None,https://www.petfinder.com/member/us/ok/bixby/o...,None,Oklahoma Westie Rescue is dedicated to the res...,None,None,None,...,None,None,NOTE: You must complete and submit an applicat...,None,None,None,None,None,None,ok151


In [29]:
# now try to drop duplicates - based on all columns
organizations2_df.drop_duplicates()
# new length is 8909

,id,name,email,phone,url,website,mission_statement,distance,address_address1,address_address2,...,hours_saturdsy,hours_sunday,adoption_policy,adoption_url,socialmedia_facebook,socialmedia_twitter,socialmedia_youtube,socialmedia_instagram,socialmedia_pinterest,organization_id
0,PA912,The Shelter Animal Project,tsapadoption@gmail.com,None,https://www.petfinder.com/member/us/pa/fairles...,None,None,None,PO Box 24,None,...,None,None,None,None,None,None,None,None,None,pa912
1,GA762,Special Touch Animal Rescue,STARRadopt@aol.com,(404) 983-3581,https://www.petfinder.com/member/us/ga/acworth...,None,We are a private foster home group. We are NOT...,None,None,None,...,None,None,None,None,None,None,None,None,None,ga762
2,TX876,Gina's Heart of Gold Reptile Rescue,gdisteldorf@comcast.net,(713) 329-9304,https://www.petfinder.com/member/us/tx/houston...,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,tx876
3,OH749,Purr-fect Companions Sanctuary,purrfectcompanions@att.net,216-671-6369,https://www.petfinder.com/member/us/oh/lakewoo...,None,None,None,P.O.Box 770992,None,...,None,None,None,None,None,None,None,None,None,oh749
4,MO556,Montgomery City Animal Shelter/Project Preciou...,projectpreciouspaws@hotmail.com,1-573-310-4979,https://www.petfinder.com/member/us/mo/montgom...,None,None,None,915 Benton Street,None,...,None,None,None,None,None,None,None,None,None,mo556
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,NJ519,Close to Home Rescue,Doggiedogrescue@aol.com,None,https://www.petfinder.com/member/us/nj/pompton...,https://www.doggiedogworld.petfinder.com,None,None,None,None,...,None,None,We are foster based and do not have a physical...,None,https://www.facebook.com/closetohome/,None,None,None,None,nj519
9996,CT477,Pack Leaders Dog Rescue of CT,packleadersrescueCT@gmail.com,None,https://www.petfinder.com/member/us/ct/manches...,http://www.packleadersrescue.com,UPCOMING ADOPTION EVENTS\r\nhttp://packleaders...,None,None,None,...,None,None,Please understand that our Rescue dogs have be...,http://packleadersrescue.com/adoption-applicat...,None,None,None,None,None,ct477
9997,KY544,"My New Kentucky Home, Inc.",mynewkentuckyhomerescue@gmail.com,(502) 509-3848,https://www.petfinder.com/member/us/ky/louisvi...,None,Our mission is to help our overcrowded Kentuck...,None,None,None,...,None,None,None,None,https://facebook.com/MyNewKentuckyHomeRescue,None,None,None,None,ky544
9998,OK151,Oklahoma Westie Rescue,adoption@okwestierescue.com,None,https://www.petfinder.com/member/us/ok/bixby/o...,None,Oklahoma Westie Rescue is dedicated to the res...,None,None,None,...,None,None,NOTE: You must complete and submit an applicat...,None,None,None,None,None,None,ok151


In [30]:
# checking number of unique names in new df
len_name_unique = len(organizations2_df["name"].unique())
len_name_unique

#so there are some names that are used twice, since there are more organization ids than names  

8885

In [31]:
# not sure we need this code: organizations.pivot_table(index=['id'], aggfunc='size')

In [32]:
organizations2_df.to_csv("organizations_mongo.csv")

In [33]:
# read csv
org_mongo_data = pd.read_csv("organizations_mongo.csv")

In [34]:
# reset index
org_mongo_data.reset_index(inplace=True)

In [35]:
org_mongo_dict = org_mongo_data.to_dict("records")

In [36]:
org_collection = db.org_collection

In [37]:
org_collection.insert_many(org_mongo_dict)

In [38]:
#for organization in org_mongo_df["organizations"]:
#    organization_dict = organization
#    organizations_collection.insert_one(animal_dict)
#    print(organization)

## Other Queries

In [39]:
# Returning data for a single animal type
# dogs = pf.animal_types('dog')